In [4]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
modello = sp.create_model(0).to(device)

batch_size = 128
epochs = 3
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=1e-2) 
early_stop = sp.EarlyStopping(2, 0.1)

data, references, descriptions = get_tabular(aggregated_month_path)
train, val = getDataset(references, data, descriptions, img_path, batch_size)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2 = sp.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    if early_stp:
        print('Early Stop attivato.')
        break
    
print("Done!")

os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_m_1')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_m_1\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati aggregati mensilmente, adamw, batch 128, 3 epoche, lr 1e-2, early stopping con 2 epoche e delta 0.01, dropout 0.5." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse}\nMAE: {mae}\nR2: {r2}")
model_sc = torch.jit.script(modello)
model_sc.save('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\agg_m_1\\weights.pt')

Sto usando cuda


C:\Users\GRVRLD00P\AppData\Local\Temp\ipykernel_1508\2784367408.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modello = torch.load('C:\\Users\\GRVRLD00P\\Documents\\P

Test Error: 
 Avg MSE: 0.186836

